In [ ]:
# default_exp dataset

In [ ]:
# hide
from ipynb_path import *

In [ ]:
# export
from counternet.import_essentials import *

Global seed set to 31


In [ ]:
# export
def dict2json(dictionary: Dict[str, Any], file_name: str):
    with open(file_name, "w") as outfile:
        json.dump(dictionary, outfile, indent=4)


def load_configs(file_name: Path):
    # if os.path.exists(file_name):
    #     raise FileNotFoundError(f"{file_name} is not found.")
    with open(file_name) as json_file:
        return json.load(json_file)


def update_json_file(param: dict, file_name: str):
    if os.path.exists(file_name):
        old_param = load_configs(file_name)
    else:
        old_param = {}
    # copy to old_param
    for k in param.keys():
        old_param[k] = param[k]
    dict2json(old_param, file_name)
    return old_param

In [ ]:
def simple_transform(df: pd.DataFrame, cat_cols: list, outcome_col: str):
    """construct features to [[--cont_features--], [--cat_features--], outcome_col]"""
    cols = df.columns.tolist()
    
    assert outcome_col in cols
    
    for col in cat_cols:
        cols.remove(col)
    cols.remove(outcome_col)
    cols += cat_cols
    cols += [outcome_col]
    return df[cols]

def simple_init_params(cols: list, cat_cols: list, outcome_col: str, file_name: str):
    # load configs of adult for no reasons
    param = load_json("assets/configs/adult.json")
    for col in cat_cols:
        cols.remove(col)
    cols.remove(outcome_col)
    # copy to cont_cols
    cont_cols = cols
    param['continous_cols'] = cont_cols
    param['cat_cols'] = cat_cols
    return update_json_file(param, file_name)

# Dummy Dataset

In [ ]:
# export
def bn_func(x1, x2, x3, x4):
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))
    return sigmoid(10.5 * ((x1 * x2) / 8100) + 10 - np.random.normal(1, 0.1, 10000) * x3 + 1e-3 * x4)


def x1_to_x3(x1):
    return 1/3 * x1 - 5


def x1x2_to_x4(x1, x2):
    return x1 * np.log(x2 ** 2) / 10 - 10


def bn_gen():
    """
    modify code from: https://github.com/divyat09/cf-feasibility/blob/master/generativecf/scripts/simple-bn-gen.py
    """
    x1 = np.random.normal(50, 15, 10000)
    x2 = np.random.normal(35, 17, 10000)
    x3 = x1_to_x3(x1) + np.random.normal(0, 1, 10000)
    x4 = x1x2_to_x4(x1, x2) + np.random.normal(0, 1, 10000)
    y = bn_func(x1, x2, x3, x4)

    data = np.zeros((x1.shape[0], 5))
    data[:, 0] = x1
    data[:, 1] = x2
    data[:, 2] = x3
    data[:, 3] = x4
    data[:, 4] = np.array(y > .5, dtype=np.int)
    return pd.DataFrame(data, columns=['x1', 'x2', 'x3', 'x4', 'y'])

# Adult Dataset

In [ ]:
# export
def load_adult_income_dataset(path=None):
    """Loads adult income dataset from https://archive.ics.uci.edu/ml/datasets/Adult and prepares the data for data analysis based on https://rpubs.com/H_Zhu/235617
    :return adult_data: returns preprocessed adult income dataset.

    copy from https://github.com/interpretml/DiCE/blob/master/dice_ml/utils/helpers.py
    """
    if path is None:
        raw_data = np.genfromtxt(
            'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',
            delimiter=', ',
            dtype=str
        )
    else:
        raw_data = np.genfromtxt(
            path,
            delimiter=', ',
            dtype=str
        )

    #  column names from "https://archive.ics.uci.edu/ml/datasets/Adult"
    column_names = ['age', 'workclass', 'fnlwgt', 'education', 'educational-num',
                    'marital-status', 'occupation', 'relationship', 'race', 'gender',
                    'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']

    adult_data = pd.DataFrame(raw_data, columns=column_names)

    # For more details on how the below transformations are made, please refer to https://rpubs.com/H_Zhu/235617
    adult_data = adult_data.astype(
        {"age": np.int64, "educational-num": np.int64, "hours-per-week": np.int64})

    adult_data = adult_data.replace(
        {'workclass': {'Without-pay': 'Other/Unknown', 'Never-worked': 'Other/Unknown'}})
    adult_data = adult_data.replace({'workclass': {
                                    'Federal-gov': 'Government', 'State-gov': 'Government', 'Local-gov': 'Government'}})
    adult_data = adult_data.replace(
        {'workclass': {'Self-emp-not-inc': 'Self-Employed', 'Self-emp-inc': 'Self-Employed'}})
    adult_data = adult_data.replace(
        {'workclass': {'Never-worked': 'Self-Employed', 'Without-pay': 'Self-Employed'}})
    adult_data = adult_data.replace({'workclass': {'?': 'Other/Unknown'}})

    adult_data = adult_data.replace({'occupation': {'Adm-clerical': 'White-Collar', 'Craft-repair': 'Blue-Collar',
                                                    'Exec-managerial': 'White-Collar', 'Farming-fishing': 'Blue-Collar',
                                                    'Handlers-cleaners': 'Blue-Collar',
                                                    'Machine-op-inspct': 'Blue-Collar', 'Other-service': 'Service',
                                                    'Priv-house-serv': 'Service',
                                                    'Prof-specialty': 'Professional', 'Protective-serv': 'Service',
                                                    'Tech-support': 'Service',
                                                    'Transport-moving': 'Blue-Collar', 'Unknown': 'Other/Unknown',
                                                    'Armed-Forces': 'Other/Unknown', '?': 'Other/Unknown'}})

    adult_data = adult_data.replace({'marital-status': {'Married-civ-spouse': 'Married',
                                                        'Married-AF-spouse': 'Married', 'Married-spouse-absent': 'Married', 'Never-married': 'Single'}})

    adult_data = adult_data.replace({'race': {'Black': 'Other', 'Asian-Pac-Islander': 'Other',
                                              'Amer-Indian-Eskimo': 'Other'}})

    adult_data = adult_data[['age', 'hours-per-week', 'workclass', 'education', 'marital-status',
                             'occupation', 'race', 'gender', 'income']]

    adult_data = adult_data.replace({'income': {'<=50K': 0, '>50K': 1}})

    adult_data = adult_data.replace({'education': {'Assoc-voc': 'Assoc', 'Assoc-acdm': 'Assoc',
                                                   '11th': 'School', '10th': 'School', '7th-8th': 'School', '9th': 'School',
                                                   '12th': 'School', '5th-6th': 'School', '1st-4th': 'School', 'Preschool': 'School'}})

    adult_data = adult_data.rename(
        columns={'marital-status': 'marital_status', 'hours-per-week': 'hours_per_week'})

    return adult_data

# OULAD

oulad data should be put under directory `data/oulad`

In [ ]:
#export
def load_learning_analytic_data(path='assets/data/oulad'):
    def weighted_score(x):
        d = {}
        total_weight = sum(x['weight'])
        d['weight'] = total_weight
        if sum(x['weight']) == 0:
            d['weighted_score'] = sum(x['score']) / len(x['score'])
        else:
            d['weighted_score'] = sum(
                x['score'] * x['weight']) / sum(x['weight'])
        return pd.DataFrame(d, index=[0])

    def clicks(x):
        types = x['activity_type']
        sum_clicks = x['sum_click']
    #     for t, c in zip(types, sum_clicks):
    #         x[f"{t}_click"] = c
        return pd.DataFrame({f"{t}_click": c for t, c in zip(types, sum_clicks)}, index=[0])

    print('loading pandas dataframes...')

    assessment = pd.read_csv(f'{path}/assessments.csv')
    courses = pd.read_csv(f'{path}/courses.csv')
    student_assessment = pd.read_csv(f'{path}/studentAssessment.csv')
    student_info = pd.read_csv(f'{path}/studentInfo.csv')
    student_regist = pd.read_csv(f'{path}/studentRegistration.csv')
    student_vle = pd.read_csv(f'{path}/studentVle.csv')
    vle = pd.read_csv(f'{path}/vle.csv')

    print('preprocessing assessment...')

    # note: only count for submitted assessment, not weighted for unsubmitted ones
    assessment_merged = student_assessment.merge(assessment)
    assessment_grouped = assessment_merged.groupby(
        ['code_module', 'code_presentation', 'id_student']).apply(weighted_score)
    assessment_df = assessment_grouped.reset_index(
        None).drop(['level_3'], axis=1)

    print('preprocessing vle...')

    # vle
    grouped_vle = student_vle.merge(vle).groupby(
        ['activity_type', 'code_module', 'code_presentation', 'id_student'])
    sumed_vle = grouped_vle.sum().drop(
        ['id_site', 'date', 'week_from', 'week_to'], axis=1).reset_index()
    grouped_vle = sumed_vle.groupby(
        ['code_module', 'code_presentation', 'id_student']).apply(clicks)
    vle_df = grouped_vle.reset_index(None).drop(['level_3'], axis=1)

    student_df = student_info.merge(assessment_df, on=['code_module', 'code_presentation', 'id_student'], how='left')\
        .merge(vle_df, on=['code_module', 'code_presentation', 'id_student'], how='left')

    return student_df[['num_of_prev_attempts', 'weight', 'weighted_score',
                       'forumng_click', 'homepage_click', 'oucontent_click',
                       'resource_click', 'subpage_click', 'url_click', 'dataplus_click',
                       'glossary_click', 'oucollaborate_click', 'quiz_click',
                       'ouelluminate_click', 'sharedsubpage_click', 'questionnaire_click',
                       'page_click', 'externalquiz_click', 'ouwiki_click', 'dualpane_click',
                       'folder_click', 'repeatactivity_click', 'htmlactivity_click',
                       'code_module', 'gender', 'region',
                       'highest_education', 'imd_band', 'age_band', 'studied_credits',
                       'disability', 'final_result']]

# HELOC

home equity line of credit
https://community.fico.com/s/explainable-machine-learning-challenge

# Default of credit card clients

http://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients

# German Credit Card Dataset

http://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data)

# Student Performance

http://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients
https://dash-xai.herokuapp.com/

# Heart

https://www.kaggle.com/andrewmvd/heart-failure-clinical-data

# Titanic

https://www.kaggle.com/ak1352/titanic-cl?select=train_cl.csv

# breast-cancer-wisconsin

# Describe Data

In [ ]:
configs = [
    ("adult", load_json("assets/configs/adult.json")),
    ("home", load_json("assets/configs/home.json")),
    ("student", load_json("assets/configs/student.json")),
    ("breast_cancer", load_json("assets/configs/extra/breast_cancer.json")),
    ("credit_card", load_json("assets/configs/extra/credit_card.json")),
    ("german_credit", load_json("assets/configs/extra/german_credit.json")),
    ("heart", load_json("assets/configs/extra/heart.json")),
    ("student_performance", load_json("assets/configs/extra/student_performance.json")),
    ("titanic", load_json("assets/configs/extra/titanic.json")),
]

In [ ]:
# export
def describe(configs: List[Dict[str, Dict[str, Any]]]):
    r = {"size": {}, "# of Cont": {}, "# of Cat": {}}
    for data_name, config in configs:
        data = pd.read_csv(f"{config['data_dir']}")
        data_size = len(data)
        cat_len = len(config['discret_cols'])
        cont_len = len(config['continous_cols'])
        r['size'][data_name] = data_size
        r['# of Cont'][data_name] = cont_len
        r['# of Cat'][data_name] = cat_len

    # pd.DataFrame.from_dict(r).to_csv("../results/data_describe.csv")
    return r

In [ ]:
describe(configs)

{'size': {'adult': 32561,
  'home': 10459,
  'student': 32593,
  'breast_cancer': 569,
  'credit_card': 30000,
  'german_credit': 1000,
  'heart': 299,
  'student_performance': 649,
  'titanic': 891},
 '# of Cont': {'adult': 2,
  'home': 21,
  'student': 23,
  'breast_cancer': 30,
  'credit_card': 20,
  'german_credit': 7,
  'heart': 7,
  'student_performance': 2,
  'titanic': 2},
 '# of Cat': {'adult': 6,
  'home': 2,
  'student': 8,
  'breast_cancer': 0,
  'credit_card': 3,
  'german_credit': 13,
  'heart': 5,
  'student_performance': 14,
  'titanic': 24}}